# ORE Dashboard

Some introduction etc...

## Launch ORE

In [ ]:
from ipywidgets import Text, Button
from IPython.display import display
import subprocess
import os

ore_path_selector = Text(description='ORE Path:', value='D:\\cppdev\\openxva\\App\\bin\\x64\\Debug\\', width=200)
xml_selector = Text(description='XML File:', value='D:\cppdev\openxva\Examples\Example_3\Input\\ore.xml', width=200)
launch_button = Button(description='Launch')

npv_cube_filename = ""

def launch_ore(b):
    cwd = os.getcwd() # save directory of the jupyter notebook
    print(cwd)
    config_path = os.path.dirname(os.path.dirname(xml_selector.value))
    os.chdir(config_path) # navigate to ore config folder
    print(os.getcwd())
    command = [ore_path_selector.value + 'ore.exe', os.path.join(os.path.join(config_path, 'Input'),'ore.xml')]
    print(command)
    print("Starting ORE run... please wait...")
    p = subprocess.Popen(command,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE)
    stdout, stderr = p.communicate()
    print(stdout)
    print(stderr)
    cwd = os.getcwd()
    os.chdir(cwd) # go back to jupyter notebook
    print("ORE run successful!")
    npv_cube_filename = os.path.join(os.path.join(config_path, 'Output'),'rawcube.csv')

launch_button.on_click(launch_ore)

display(ore_path_selector)
display(xml_selector)
display(launch_button)

In [ ]:
print(os.getcwd())

## Select Netting Sets and Trades

In [ ]:
print(npv_cube_filename)

import pandas as pd       

from ipywidgets import Select, SelectMultiple, HBox

current_netting_sets = [0]
def set_current_netting_sets(netting_sets):
    current_netting_sets = netting_sets['new']

current_trades = [0]
def set_current_trades(trades):
    current_trades = trades['new']

netting_set_selector = Select(description='Netting Sets:')
trades_selector = SelectMultiple(description='Trades:')
netting_set_selector.observe(set_current_netting_sets, names='value')
trades_selector.observe(set_current_trades, names='value')
def load_cube_file(b):
    csv_file = npv_file_selector.value
    global df
    df = pd.read_csv(csv_file)
    df.columns = ['Id', 'NettingSet', 'DateIndex' , 'Date', 'Sample', 'Value']
    netting_set_selector.options = list(df.NettingSet.unique())
    trades_selector.options = list(df.Id.unique())

npv_file_selector = Text(description='Cube File:', value='D:\cppdev\openxva\Examples\Example_2\Output\\rawcube.csv', width=200)
read_npv_button = Button(description='Load')
read_npv_button.on_click(load_cube_file)

display(HBox([npv_file_selector, read_npv_button]))
display(HBox([netting_set_selector, trades_selector]))

## Configure Exposure Statistics

In [ ]:
import numpy as np

from ipywidgets import RadioButtons, FloatSlider

%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from enum import Enum
from matplotlib.font_manager import FontProperties

class ExposureTypes(Enum):
    EE = 'EE'
    EPE = 'EPE'
    ENE = 'ENE'
    PFE = 'PFE'
    
exposure_selector = RadioButtons(description='Exposure Type:', options=[member.value for name,member in ExposureTypes.__members__.items()])
percentile_selector = FloatSlider(min=90, max=100, value=95, description='PFE Percentile:')

display(HBox([exposure_selector, percentile_selector]))

## Plot Exposure Surface

In [ ]:
from scipy.stats.kde import gaussian_kde
from numpy.linalg.linalg import LinAlgError

# sum values over selected trades in netting set
df_surface = df[(df.NettingSet == netting_set_selector.value) & df.Id.isin(trades_selector.value)][['Id', 'Value', 'Sample', 'DateIndex']].groupby(['DateIndex','Sample']).sum().reset_index()
# truncate for EPE or ENE
if ExposureTypes(exposure_selector.value) == ExposureTypes.EPE:
    df_surface.loc[df_surface['Value']<0,'Value']=0
elif ExposureTypes(exposure_selector.value) == ExposureTypes.ENE:
    df_surface.loc[df_surface['Value']>0,'Value']=0

# for each date index, calculate densities of distribution of values
grid_size = 50
npv_min = df_surface.Value.min()
npv_max = df_surface.Value.max()
dist_space = np.linspace(npv_min, npv_max, grid_size)
num_dates = len(df_surface.DateIndex.unique())
density_values = np.zeros((num_dates, grid_size))
for k in range(num_dates):
    row = df_surface.loc[df_surface['DateIndex']==k, 'Value'].values
    try:
        density = gaussian_kde(row)
        density_values[k] = density(dist_space)
    except:
        density_values[k] = np.zeros(grid_size)
density_max = np.max(density_values)

fig_surface = plt.figure()
ax_surface =fig_surface.add_subplot(111, projection='3d')

date_step = 5
#ax_surface.set_xticks(np.arange(0, num_dates, date_step))
#ax_surface.set_xticklabels(npv.dates[0::date_step])
#ax_surface.set_xlabel('Dates')
#ax_surface.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
#ax_surface.set_ylabel(current_exposure.value)

X, Y = np.meshgrid(np.arange(num_dates), dist_space)
ax_surface.plot_trisurf(X.flatten(), Y.flatten(), density_values.T.flatten(), cmap=cm.jet, linewidth=0)
plt.show()

## Time Slider

In [ ]:
from ipywidgets import IntSlider

fig_surface = plt.figure()
ax_time_slider = fig_surface.add_subplot(111)
fig_surface.canvas.set_window_title('Time Slider')
            
def plot_time_slider(change):
    date = change['new']
    plt.cla()
    ax_time_slider.plot(dist_space, density_values[date], color='k', label=exposure_selector.value)
    ax_time_slider.set_xlim(npv_min, npv_max)
    ax_time_slider.set_ylim(0, density_max)
    
time_slider = IntSlider(min=0, max=num_dates, value=0, description='DateIndex:')
time_slider.observe(plot_time_slider, names='value')
display(time_slider)


## Exposure Statistics

In [ ]:
df_stats = df[(df.NettingSet == netting_set_selector.value) & df.Id.isin(trades_selector.value)][['Id', 'Value', 'Sample', 'DateIndex']].groupby(['DateIndex','Sample']).sum().reset_index()
df_epe = df_stats.copy()
df_ene = df_stats.copy()

df_epe.loc[df_epe['Value']<0,'Value']=0
df_ene.loc[df_ene['Value']>0,'Value']=0

ee = df_stats[['DateIndex', 'Value']].groupby('DateIndex').mean().values[:,0]
epe = df_epe[['DateIndex', 'Value']].groupby('DateIndex').mean().values[:,0]
ene = df_ene[['DateIndex', 'Value']].groupby('DateIndex').mean().values[:,0]
pfe = df_stats[['DateIndex', 'Value']].groupby('DateIndex').quantile(0.95).values[:,0]

In [ ]:
fig_statistics = plt.figure()
ax_statistics = fig_statistics.add_subplot(111)
ax_statistics.plot(range(num_dates), ee, label=ExposureTypes.EE.value)
ax_statistics.plot(range(num_dates), epe, label=ExposureTypes.EPE.value)
ax_statistics.plot(range(num_dates), ene, label=ExposureTypes.ENE.value)
ax_statistics.plot(range(num_dates), pfe, label=ExposureTypes.PFE.value)
ax_statistics.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
date_step = 2
ax_statistics.set_xticks(np.arange(0, num_dates, date_step))
#x_statistics.set_xticklabels(npv.dates[0::date_step], rotation=90)
fontP = FontProperties() # set legend outside plot
fontP.set_size('small')
ax_statistics.legend(loc='upper right', shadow=True, prop=fontP)
plt.show()

In [ ]:
import bqplot as bq
xs = bq.LinearScale()
ys = bq.LinearScale()
x = range(num_dates)
y = np.vstack((ee,epe,ene,pfe))

line = bq.Lines(x=x, y=y, scales={'x': xs, 'y': ys}, display_legend=True, labels=['EE','EPE', 'ENE', 'PFE'])
xax = bq.Axis(scale=xs, label='time', grid_lines='solid')
yax = bq.Axis(scale=ys, orientation='vertical', tick_format='0.2f', label='exposure', grid_lines='solid')

fig = bq.Figure(marks=[line], axes=[xax, yax], animation_duration=1000)
display(fig)